#Price scraping


In [ ]:
#import 
import string 
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import date,datetime

In [ ]:
#Function for Scraping Data
def ScrapingData(url):
    k = requests.get(url).text
    soup=BeautifulSoup(k,'html.parser')  
    productlist = soup.find_all("div",{"class":"pagination_pagination__RUsgT"})
    if not productlist:
        page = 1
    else:
        page = productlist[0].find_all("a")
        page = page[-1].find("span").text

    name = []
    link = []
    base_price = []
    price = []
    prefix_link = url.replace('https://','')
    prefix_link = prefix_link.replace('http://','')
    prefix_link = prefix_link.split('/')[0]

    for i in range(1,int(page)+1):
        CurrentURL = url + "?page=" + str(i)
        k = requests.get(CurrentURL).text
        soup=BeautifulSoup(k,'html.parser')  
        productlist = soup.find_all("div",{"class":"category_result_row__qfMRp"})

        base_prices = productlist[0].find_all("span",{"class":"productCard_base_price__j5_Sq"})
        prices = productlist[0].find_all("div",{"class":"productCard_price__zid5Z"})
        products = productlist[0].find_all("div",{"class":"productCard_title__HUSZQ"})

        for i,j,k in zip(products,base_prices,prices):
            name.append(i.find('a').text)
            link.append(prefix_link + i.find("a").get('href'))
            p = k.text.split('/')
            price.append(p[0])
            if (not j.text):
                base_price.append(p[0])
            else:
                base_price.append(j.text)
    output = {}
    output['name'] = name
    output['base_price'] = base_price
    output['price'] = price
    output['link'] = link
    output['webname'] = prefix_link.split('.')[1]
    return output

In [ ]:
#function scrap specific data
def ScrapingSpecificData(link):
    if (link.find("http://") == -1) & (link.find("https://") == -1):
        link = 'http://'+link
    k = requests.get(link).text
    soup=BeautifulSoup(k,'html.parser')
    name = soup.find_all('h1',{'class':'productDetail_product_title__fb0ow'})[0].text
    pack_detail = soup.find_all('span',{"class":"productDetail_unit__QntiB"})
    detail = soup.find_all("div",{"class":"productDetail_category__mbviF"})
    for i in string.punctuation:
        name = name.replace(i,'')
    name = name.split(' ')
    output = {}

    if not pack_detail:
        pack = ''
    else:
        pack = pack_detail[0].text[2:]
    for n in name:
        if (n.isdigit()) & (not name[-1].isdigit()):
            pack = name[-1]

    if ('กก' in pack) | ('กิโลกรัม' in pack):
        pack = 'KG'
    elif ('ก' in pack) | ('กรัม' in pack):
        pack = 'G'
    output['Pack'] = pack

    if (len(name) > 2):
        if name[-2].isdigit():
            output['Unit'] = name[-2]
        else:
            output['Unit'] = 1
    else:
        output['Unit'] = 1

    if not detail:
        brand = ''
        category = ''
    else:
        brand = (detail[0].find("b"))
        category = (detail[0].find("a"))
    if not brand:
        output['Brand'] = ''
    else:
        output['Brand'] = brand.text
    if not category:
        output['Product'] = ''
    else:
        output['Product'] = category.text

    return output

In [ ]:
#get data 
dic = {'Date':[],
       'Product':[],
       'Product Group':[],
       'Store':[],
       'Product Name':[],
       'Brand':[],
       'Pack':[],
       'Unit':[],
       'Original Price':[],
       'Final Price':[],
       'url':[]}

group = ['หมู','ไก่','marinated','ไส้กรอก','แฮม','บาโลน่า','เบคอน','ลูกชิ้น','เนื้อสัตว์หมักปรุงรส']

URLs = pd.read_csv('BigC_Input_URL.csv') #open file URL here
   
for url in URLs.url:
    x = ScrapingData(url)
    dic['Product Name'].extend(x['name'])
    dic['Original Price'].extend(x['base_price'])
    dic['Final Price'].extend(x['price'])
    dic['url'].extend(x['link'])

for link in dic['url']:
    y = ScrapingSpecificData(link)
    dic['Date'].append(datetime.now().strftime('%d-%m-%y %H:%M'))
    dic['Store'].append('Big-C')
    dic['Product'].append(y['Product'])
    dic['Brand'].append(y['Brand'])
    dic['Pack'].append(y['Pack'])
    dic['Unit'].append(str(y['Unit']))
    selct_group = ''
    for g in group:
        if g in y['Product']:
            selct_group = g
            break
    dic['Product Group'].append(selct_group)

In [ ]:
#write to excel
df = pd.DataFrame(dic)
df.to_csv(x['webname']+'_Ouput_'+str(date.today())+'.csv',index=False)